In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# # Columns identified for removal
# DROP_COLUMNS = ['SpecialReduction','SpecialRehabilitation','SpecialMedications','Driver Age',
#                 'Gender','Accident Date','Claim Date', 'Accident Description', 'Injury Description']

# # columns to convert to binary
# BINARY_COLUMNS = ['Exceptional_Circumstances','Minor_Psychological_Injury','Whiplash',
#                   'Police Report Filed','Witness Present']

# # categorical columns for one-hot encoding
# CATEGORY_COLUMNS = ['Dominant injury','Vehicle Type','Weather Conditions','AccidentType']

# # numerica data columns
# NUMERIC_COLUMNS = ['SpecialHealthExpenses','SpecialOverage','GeneralRest',
#                     'SpecialAdditionalInjury','SpecialEarningsLoss','SpecialUsageLoss',
#                     'SpecialAssetDamage','SpecialFixes','GeneralFixed','GeneralUplift',
#                     'SpecialLoanerVehicle','SpecialTripCosts','SpecialJourneyExpenses','SpecialTherapy']

# Columns identified for removal
DROP_COLUMNS = ['Accident Date','Claim Date',]

# columns to convert to binary
BINARY_COLUMNS = ['Exceptional_Circumstances','Minor_Psychological_Injury','Whiplash',
                  'Police Report Filed','Witness Present']

# categorical columns for one-hot encoding
CATEGORY_COLUMNS = ['Dominant injury','Vehicle Type','Weather Conditions','Gender',
                    'AccidentType','Number of Passengers', 'Accident Description', 'Injury Description']

# numerica data columns
NUMERIC_COLUMNS = ['SpecialReduction','SpecialRehabilitation','SpecialMedications',
                   'SpecialHealthExpenses','SpecialOverage','GeneralRest','Driver Age',
                    'SpecialAdditionalInjury','SpecialEarningsLoss','SpecialUsageLoss',
                    'SpecialAssetDamage','SpecialFixes','GeneralFixed','GeneralUplift',
                    'SpecialLoanerVehicle','SpecialTripCosts','SpecialJourneyExpenses','SpecialTherapy']

IMPUTE_ZERO_COLUMNS = ['SpecialReduction','SpecialRehabilitation','SpecialMedications',
                   'SpecialHealthExpenses','SpecialOverage','GeneralRest',
                    'SpecialAdditionalInjury','SpecialEarningsLoss','SpecialUsageLoss',
                    'SpecialAssetDamage','SpecialFixes','GeneralUplift','SpecialTripCosts',
                    'SpecialLoanerVehicle','SpecialJourneyExpenses','SpecialTherapy']

# columns that require special handling
SPECIAL_COLUMN = ['Injury_Prognosis',]

# target value
TARGET_COLUMN = 'SettlementValue'

def extract_int_from_string(df, col):
    df[col] = (
        df[col]
        .str.extract('(\d+)', expand=False)
        .astype('Int64')
    )
    return df


def binary_encode(df, columns, positive_value):
    for column in columns:
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
        df[column] = df[column].astype('Int8')
    return df


def one_hot_encode(df, columns):
    for column in columns:
        df = pd.concat([df, pd.get_dummies(df[column], prefix=column)], axis=1)
        df = df.drop(column, axis=1)
    return df


def float_columns_to_int(df):
    for column in df.select_dtypes(include='float64'):
        df[column] = df[column].round().astype('Int64')
    return df


def zero_fill_num_columns(df):
    for column in df.select_dtypes(include='number'):
        df[column] = df[column].fillna(0)
    return df

# inserts string 'unknown' into category columns
def fill_category_columns(df):
    for column in df.select_dtypes(include='object'):
        df[column] = df[column].fillna('Unknown')
    return df

def preprocess_data(ml_dataset):
    df = ml_dataset.copy()
    df = df.dropna(subset=['SettlementValue'])
    df = df.drop(DROP_COLUMNS, axis=1)
    df = extract_int_from_string(df,'Injury_Prognosis')
    df = df.drop(DROP_COLUMNS, axis=1)
    df = df.drop(CATEGORY_COLUMNS, axis=1)
    df = df.dropna(subset=['SettlementValue','Injury_Prognosis','Exceptional_Circumstances','Whiplash'])
    df['Number of Passengers'] = df['Number of Passengers'].fillna(1)
    df = extract_int_from_string(df,'Injury_Prognosis')
    df = zero_fill_num_columns(df)
    df = fill_category_columns(df)
    df = binary_encode(df,BINARY_COLUMNS,'Yes')
    df = one_hot_encode(df,CATEGORY_COLUMNS)
    df = float_columns_to_int(df)
    scaler = RobustScaler()
    df[NUMERIC_COLUMNS] = scaler.fit_transform(df[NUMERIC_COLUMNS])
    print("Number of records remaining: " + str(len(df.index)))
    return df

In [74]:
# import dataset from csv
ml_dataset = pd.read_csv('./dataset-excel-cleaned.csv')

# confirm number of imported records matches expected size
print("Number of records imported: " + str(len(ml_dataset.index)))

Number of records imported: 4894


In [75]:
import janitor
clean_df = ml_dataset.clean_names().remove_empty()
for column in clean_df.columns:
    if "_" in column:
        new_column = column.replace("_", "")
        clean_df.rename(columns={column: new_column}, inplace=True)
duplicates = clean_df.get_dupes()
if len(duplicates.index) > 0:
    clean_df = clean_df[~duplicates]
print("Records remaining after initial processing: " + str(len(clean_df.index)))

Records remaining after initial processing: 4894


In [76]:
# populate missing values in the 'accidentdate' and 'claimdate' columns
clean_df['accidentdate'] = clean_df['accidentdate'].fillna(clean_df['claimdate'])
clean_df['claimdate'] = clean_df['claimdate'].fillna(clean_df['accidentdate'])
clean_df = clean_df.dropna(subset=['accidentdate','claimdate'])
print("Records remaining after date processing: " + str(len(clean_df.index)))

Records remaining after date processing: 4887


In [77]:
# Limit numberofpassengers to 2 for motorbikes
clean_df.loc[clean_df['vehicletype'] == 'Motorcycle', 'numberofpassengers'] = clean_df.loc[clean_df['vehicletype'] == 'Motorcycle', 'numberofpassengers'].clip(upper=2)

In [78]:
# Fill missing values in 'generalfixed' with 'settlementvalue' if it matches distinct values in 'generalfixed'
distinct_values = clean_df['generalfixed'].dropna().unique()
clean_df.loc[clean_df['generalfixed'].isna() & clean_df['settlementvalue'].isin(distinct_values), 'generalfixed'] = clean_df['settlementvalue']

In [79]:
# create bins for injury prognosis based on the whiplash tariff scale
import numpy as np
bin_edges = [0, 3, 6, 9, 12, 15, 18, 24, np.inf]
clean_df = clean_df.bin_numeric(from_column_name='injuryprognosis',
                    to_column_name='prognosisgroup',
                    bins=bin_edges,
                    right=True)

In [80]:
# Fill missing 'generalfixed' values based on 'prognosisgroup'
clean_df['generalfixed'] = clean_df.apply(
    lambda row: clean_df.loc[
        (clean_df['prognosisgroup'] == row['prognosisgroup']) & 
        (clean_df['generalfixed'].notna()), 'generalfixed'
    ].iloc[0] if pd.isna(row['generalfixed']) and 
    clean_df.loc[
        (clean_df['prognosisgroup'] == row['prognosisgroup']) & 
        (clean_df['generalfixed'].notna())
    ].shape[0] > 0 else row['generalfixed'], axis=1
)

In [81]:
# Fill missing prognosisgroup using generalfixed matches
clean_df['prognosisgroup'] = clean_df.apply(
    lambda row: clean_df.loc[
        (clean_df['generalfixed'] == row['generalfixed']) & 
        (clean_df['prognosisgroup'].notna()), 'prognosisgroup'
    ].iloc[0] if pd.isna(row['prognosisgroup']) and 
    clean_df.loc[
        (clean_df['generalfixed'] == row['generalfixed']) & 
        (clean_df['prognosisgroup'].notna())
    ].shape[0] > 0 else row['prognosisgroup'], axis=1
)

In [83]:
# remaining rows with missing value in 'generalfixed' are dropped as it 
# not possible to reliably impute these values based on the available data
clean_df = clean_df.dropna(subset=['generalfixed'])
print("Records remaining after generalfixed processing: " + str(len(clean_df.index)))

Records remaining after generalfixed processing: 4878


In [ ]:
clean_df.head(20)



,settlementvalue,injuryprognosis,injurydescription,dominantinjury,whiplash,minorpsychologicalinjury,exceptionalcircumstances,generalfixed,generaluplift,generalrest,...,weatherconditions,vehicleage,driverage,numberofpassengers,policereportfiled,witnesspresent,gender,accidentdate,claimdate,prognosisgroup
0,520,5.0,Whiplash and minor bruises.,Arms,1.0,1.0,0.0,520.0,0.0,0.0,...,Rainy,13.0,33.0,2,1,1,Male,10/11/2023 11:22,11/06/2024 11:22,"(3.0, 6.0]"
1,870,2.0,Minor cuts and scrapes.,Multiple,1.0,1.0,0.0,260.0,0.0,520.0,...,Snowy,4.0,45.0,2,1,1,Female,25/06/2023 00:55,09/01/2024 00:55,"(0.0, 3.0]"
2,2140,7.0,Whiplash and minor bruises.,Legs,1.0,0.0,0.0,840.0,0.0,1400.0,...,Sunny,9.0,45.0,2,1,0,Female,23/02/2020 17:43,01/03/2020 17:43,"(6.0, 9.0]"
3,520,4.0,Minor cuts and scrapes.,Arms,1.0,1.0,0.0,520.0,0.0,0.0,...,Rainy,5.0,62.0,1,1,1,Female,02/10/2021 04:36,13/10/2021 04:36,"(3.0, 6.0]"
4,260,3.0,Concussion and bruised ribs.,Arms,0.0,1.0,0.0,260.0,0.0,0.0,...,Rainy,9.0,78.0,1,1,1,Other,02/04/2023 05:13,14/04/2023 05:13,"(0.0, 3.0]"
5,520,6.0,Minor cuts and scrapes.,Multiple,1.0,1.0,0.0,520.0,0.0,0.0,...,Rainy,10.0,55.0,1,1,0,Other,12/03/2020 13:18,30/06/2020 13:18,"(3.0, 6.0]"
6,1015,NaN,Whiplash and minor bruises.,Legs,1.0,1.0,0.0,520.0,0.0,0.0,...,Sunny,5.0,72.0,1,0,0,Female,08/04/2023 08:24,19/08/2023 08:24,"(3.0, 6.0]"
7,1032,8.0,Concussion and bruised ribs.,Legs,1.0,0.0,0.0,840.0,0.0,0.0,...,Sunny,9.0,77.0,4,0,1,Female,14/10/2021 11:00,19/02/2022 11:00,"(6.0, 9.0]"
8,808,6.0,Minor cuts and scrapes.,Arms,0.0,1.0,0.0,520.0,0.0,0.0,...,Snowy,13.0,23.0,2,1,1,Female,09/09/2020 15:07,04/02/2021 15:07,"(3.0, 6.0]"
9,500,2.0,Minor cuts and scrapes.,Hips,1.0,1.0,0.0,260.0,0.0,0.0,...,Rainy,19.0,59.0,2,1,1,Female,01/09/2021 12:37,25/10/2021 12:37,"(0.0, 3.0]"


In [ ]:
# Fill missing injuryprognosis with midpoint of prognosisgroup range
imputed_prognosis = clean_df.copy()

# Fill missing injuryprognosis with midpoint of prognosisgroup range
for idx, row in clean_df[pd.isna(clean_df['injuryprognosis'])].iterrows():
    if hasattr(row['prognosisgroup'], 'left') and hasattr(row['prognosisgroup'], 'right'):
        clean_df.at[idx, 'injuryprognosis'] = ((row['prognosisgroup'].left + row['prognosisgroup'].right) / 2).round(0)

# Verify the number of remaining null values in injuryprognosis
print(f"Remaining null values in injuryprognosis: {clean_df['injuryprognosis'].isna().sum()}")


# clean_df['injuryprognosis'] = clean_df.apply(
#     lambda row: clean_df.loc[
#         (clean_df['generalfixed'] == row['generalfixed']) & 
#         (clean_df['prognosisgroup'].notna()), 'prognosisgroup'
#     ].iloc[0] if pd.isna(row['prognosisgroup']) and 
#     clean_df.loc[
#         (clean_df['generalfixed'] == row['generalfixed']) & 
#         (clean_df['prognosisgroup'].notna())
#     ].shape[0] > 0 else row['prognosisgroup'], axis=1

Remaining null values in injuryprognosis: 0


In [67]:
from sklearn.impute import KNNImputer
# columns used for KNN imputation
# KNN_COLUMNS = ['settlementvalue','injuryprognosis','prognosisgroup','exceptionalcircumstances','minorpsychologicalinjury','whiplash','generalfixed','generaluplift']
KNN_COLUMNS = ['injuryprognosis','generalfixed']
knn_subset = clean_df[KNN_COLUMNS].copy()
knn_subset.head()

,injuryprognosis,generalfixed
0,5.0,520.0
1,2.0,260.0
2,7.0,840.0
3,4.0,520.0
4,3.0,260.0


In [68]:
imputer = KNNImputer(n_neighbors=5, weights='distance')
imputed_np = imputer.fit_transform(knn_subset)
imputed_df = pd.DataFrame(imputed_np, columns=KNN_COLUMNS)
imputed_df = imputed_df.round(0)
imputed_df.head()

,injuryprognosis,generalfixed
0,5.0,520.0
1,2.0,260.0
2,7.0,840.0
3,4.0,520.0
4,3.0,260.0


In [69]:
for col in KNN_COLUMNS:
    clean_df[col] = imputed_df[col]
clean_df.head()

,settlementvalue,injuryprognosis,injurydescription,dominantinjury,whiplash,minorpsychologicalinjury,exceptionalcircumstances,generalfixed,generaluplift,generalrest,...,weatherconditions,vehicleage,driverage,numberofpassengers,policereportfiled,witnesspresent,gender,accidentdate,claimdate,prognosisgroup
0,520,5.0,Whiplash and minor bruises.,Arms,1.0,1.0,0.0,520.0,0.0,0.0,...,Rainy,13.0,33.0,2,1,1,Male,10/11/2023 11:22,11/06/2024 11:22,"(3.0, 6.0]"
1,870,2.0,Minor cuts and scrapes.,Multiple,1.0,1.0,0.0,260.0,0.0,520.0,...,Snowy,4.0,45.0,2,1,1,Female,25/06/2023 00:55,09/01/2024 00:55,"(0.0, 3.0]"
2,2140,7.0,Whiplash and minor bruises.,Legs,1.0,0.0,0.0,840.0,0.0,1400.0,...,Sunny,9.0,45.0,2,1,0,Female,23/02/2020 17:43,01/03/2020 17:43,"(6.0, 9.0]"
3,520,4.0,Minor cuts and scrapes.,Arms,1.0,1.0,0.0,520.0,0.0,0.0,...,Rainy,5.0,62.0,1,1,1,Female,02/10/2021 04:36,13/10/2021 04:36,"(3.0, 6.0]"
4,260,3.0,Concussion and bruised ribs.,Arms,0.0,1.0,0.0,260.0,0.0,0.0,...,Rainy,9.0,78.0,1,1,1,Other,02/04/2023 05:13,14/04/2023 05:13,"(0.0, 3.0]"


In [70]:
# Fill missing values in generaluplift and exceptionalcircumstances
clean_df.loc[
    (clean_df['settlementvalue'] == clean_df['generalfixed']) & 
    (clean_df[['generaluplift', 'exceptionalcircumstances']].isnull().any(axis=1)),
    ['generaluplift', 'exceptionalcircumstances']
] = clean_df[['generaluplift', 'exceptionalcircumstances']].fillna(0)

In [71]:
# where settlement and generalfixed are equal indicates a whiplash claim
# and the whiplash column should be filled with 1
clean_df.loc[
    (clean_df['settlementvalue'] == clean_df['generalfixed']) & 
    (clean_df[['whiplash']].isnull().any(axis=1)),
    ['whiplash']
] = clean_df[['whiplash']].fillna(1)

In [72]:
# where settlement and generalfixed are equal indicates a whiplash claim
# and the whiplash column should be filled with 1
clean_df.loc[pd.isna(clean_df['generaluplift']) & (clean_df['exceptionalcircumstances'] == 0), 'generaluplift'] = 0
clean_df.loc[pd.isna(clean_df['generaluplift']) & (clean_df['exceptionalcircumstances'] == 1), 'generaluplift'] = clean_df.loc[pd.isna(clean_df['generaluplift']) & (clean_df['exceptionalcircumstances'] == 1), 'generalfixed'] * 0.2

In [73]:
# Impute zeros for specific columns if settlementvalue and generalfixed match
IMPUTE_ZERO_COLUMNS = [
    'specialreduction', 'specialrehabilitation', 'specialmedications',
    'specialhealthexpenses', 'specialoverage', 'generalrest',
    'specialadditionalinjury', 'specialearningsloss', 'specialusageloss',
    'specialassetdamage', 'specialfixes', 'generaluplift', 'specialtripcosts',
    'specialloanervehicle', 'specialjourneyexpenses', 'specialtherapy'
]
# Filter rows where settlementvalue equals generalfixed
matching_rows = clean_df['settlementvalue'] == clean_df['generalfixed']
# Impute zeros for missing values in the specified columns
clean_df.loc[matching_rows, IMPUTE_ZERO_COLUMNS] = clean_df.loc[matching_rows, IMPUTE_ZERO_COLUMNS].fillna(0)